In [1]:
import gradio as gr
import anthropic
import os
import json
from dotenv import load_dotenv


In [2]:
# --- Configuration and Setup ---
load_dotenv()
# --- IMPORTANT: Configure the Anthropic API Key ---
# It's best practice to set this as an environment variable
try:
    ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
    if not ANTHROPIC_API_KEY:
        raise ValueError("ANTHROPIC_API_KEY environment variable not set.")
    # Initialize the Anthropic client
    client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
except Exception as e:
    print(f"Error configuring Anthropic API: {e}")
    # Exit if the API key isn't configured, as the app cannot run.
    exit()


In [3]:
# The absolute path to the root directory the LLM is allowed to access.
ALLOWED_DIRECTORY = os.path.expanduser("~/projects/pleasurewebsite/mysite")

if not os.path.isdir(ALLOWED_DIRECTORY):
    print(f"Error: The specified directory '{ALLOWED_DIRECTORY}' does not exist.")
    exit()

# --- Tool Functions (These remain the same) ---


In [4]:
# --- Tool Functions (These remain the same) ---

def list_files_in_directory(path: str = "."):
    """
    Lists files and directories within a specified path inside the allowed directory.
    The path is relative to the project root.
    """
    try:
        base_path = ALLOWED_DIRECTORY
        requested_path = os.path.abspath(os.path.join(base_path, path))
        if not requested_path.startswith(base_path):
            return {"error": "Access denied."}
        if not os.path.isdir(requested_path):
            return {"error": f"'{path}' is not a valid directory."}
        items = os.listdir(requested_path)
        dirs = sorted([d for d in items if os.path.isdir(os.path.join(requested_path, d))])
        files = sorted([f for f in items if os.path.isfile(os.path.join(requested_path, f))])
        
        response_str = f"Contents of '{path}':\n\n"
        if dirs:
            response_str += "Directories:\n" + "\n".join(f"- {d}/" for d in dirs) + "\n\n"
        if files:
            response_str += "Files:\n" + "\n".join(f"- {f}" for f in files) + "\n"
        
        return {"content": response_str if items else f"The directory '{path}' is empty."}
    except Exception as e:
        return {"error": f"An error occurred: {str(e)}"}

In [5]:
def read_file_content(file_path: str):
    """
    Reads the text content of a specific file within the allowed directory.
    The file_path is relative to the project root.
    """
    try:
        base_path = ALLOWED_DIRECTORY
        requested_path = os.path.abspath(os.path.join(base_path, file_path))
        if not requested_path.startswith(base_path):
            return {"error": "Access denied."}
        if not os.path.isfile(requested_path):
            return {"error": f"File not found at '{file_path}'"}
        with open(requested_path, 'r', encoding='utf-8', errors='ignore') as f:
            content = f.read(5000)
            final_content = content + "\n\n[... file content truncated ...]" if len(content) == 5000 else content
            return {"content": final_content}
    except Exception as e:
        return {"error": f"An error occurred while reading the file: {str(e)}"}


In [6]:

# --- Core Chat Logic (Rewritten for Anthropic's Claude) ---

def chat_with_tools(user_message, history):
    """
    Handles the conversation using Anthropic's Claude model, including tool calls and streaming.
    """
    system_prompt = f"You are a helpful AI assistant with access to a local Django project file system. The root of the project is '{ALLOWED_DIRECTORY}'. You can list files and read their content. When asked about the project, use your tools to find information before answering. Be concise."
    
    # A dictionary to map function names (strings) to the actual functions
    available_functions = {
        "list_files_in_directory": list_files_in_directory,
        "read_file_content": read_file_content,
    }
    
    # Define the tools in the format required by the Anthropic API
    tools = [
        {
            "name": "list_files_in_directory",
            "description": "Lists files and directories within a specified path inside the allowed directory.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "path": {
                        "type": "string",
                        "description": "The relative path to the directory. Defaults to the root if not provided."
                    }
                },
                "required": ["path"]
            }
        },
        {
            "name": "read_file_content",
            "description": "Reads the text content of a specific file within the allowed directory.",
            "input_schema": {
                "type": "object",
                "properties": {
                    "file_path": {
                        "type": "string",
                        "description": "The relative path to the file."
                    }
                },
                "required": ["file_path"]
            }
        }
    ]

    # Convert Gradio history to Claude's message format
    messages = []
    for human_msg, ai_msg in history:
        messages.append({"role": "user", "content": human_msg})
        messages.append({"role": "assistant", "content": ai_msg})
    messages.append({"role": "user", "content": user_message})

    try:
        # First API call to see if the model wants to use a tool
        response = client.messages.create(
            model="claude-3-haiku-20240307", # Or another Claude model
            system=system_prompt,
            max_tokens=1024,
            messages=messages,
            tools=tools,
            tool_choice={"type": "auto"}
        )

        if response.stop_reason == "tool_use":
            # The model wants to use a tool
            tool_use = next(block for block in response.content if block.type == "tool_use")
            tool_name = tool_use.name
            tool_input = tool_use.input
            tool_use_id = tool_use.id

            print(f"🤖 Calling tool: {tool_name}({tool_input})")
            
            # Call the actual Python function
            tool_result = available_functions[tool_name](**tool_input)

            # Append the assistant's request and the tool's result to the message history
            messages.append({"role": "assistant", "content": response.content})
            messages.append({
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use_id,
                        "content": json.dumps(tool_result),
                    }
                ]
            })

            # Second API call to get the final, user-facing response (streaming)
            with client.messages.stream(
                model="claude-3-haiku-20240307",
                system=system_prompt,
                max_tokens=1024,
                messages=messages,
            ) as stream:
                for chunk in stream.text_stream:
                    yield chunk
        else:
            # No tool use, yield the direct text response
            text_content = next((block.text for block in response.content if block.type == 'text'), None)
            if text_content:
                yield text_content

    except Exception as e:
        print(f"An error occurred in the Claude chat logic: {e}")
        yield "Sorry, an error occurred with the AI model. Please check the console for details."


In [7]:

# --- Gradio Interface (This part remains the same) ---

with gr.Blocks(theme="soft", title="Django Project Assistant (Claude) 🤖") as demo:
    gr.Markdown("# Django Project Assistant (with Claude) 🤖")
    gr.Markdown("Ask me questions about your Django project. I can list files and read their contents to help you.")
    
    chatbot = gr.Chatbot(label="Conversation", height=600, bubble_full_width=False) 
    
    with gr.Row():
        msg_textbox = gr.Textbox(
            label="Your Message",
            placeholder="e.g., What files are in mysite/?",
            scale=7,
            autofocus=True,
        )
        submit_btn = gr.Button("Send", variant="primary", scale=1)

    def handle_chat(user_input, history):
        history.append([user_input, ""])
        response_stream = chat_with_tools(user_input, history[:-1])
        for chunk in response_stream:
            history[-1][1] += chunk
            yield history, ""
        yield history, ""

    submit_btn.click(handle_chat, [msg_textbox, chatbot], [chatbot, msg_textbox])
    msg_textbox.submit(handle_chat, [msg_textbox, chatbot], [chatbot, msg_textbox])

    gr.Examples(
        examples=[
            ["What files are in the root directory?"],
            ["Can you show me the contents of 'mysite/settings.py'?"],
            ["What's inside the 'home/templates/home' directory?"]
        ],
        inputs=msg_textbox,
        outputs=[chatbot, msg_textbox],
        fn=handle_chat,
        cache_examples=False,
    )



/tmp/ipykernel_340387/897925193.py:7: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", height=600, bubble_full_width=False)
/tmp/ipykernel_340387/897925193.py:7: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(label="Conversation", height=600, bubble_full_width=False)


In [ ]:
if __name__ == "__main__":
    demo.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


🤖 Calling tool: list_files_in_directory({'path': '/home/pradeep/projects/pleasurewebsite/mysite'})
🤖 Calling tool: read_file_content({'file_path': 'mysite/settings.py'})
🤖 Calling tool: list_files_in_directory({'path': 'home/templates/home'})
